 PDF 분할 작업을 위해 `unstructured`를 사용한다`unstructured` 를 위해 다음과 도구의 설치가 필요하다

  - tesseract : 광학 문자 인식(OCR)을 위해 사용
  - poppler : PDF 렌더링 및 처리

  [poppler 설치 방법](https://pdf2image.readthedocs.io/en/latest/installation.html)과 [tesseract 설치 방법](https://tesseract-ocr.github.io/tessdoc/Installation.html)을 참고

# **라이브러리 설치**

In [ ]:
%%capture --no-stderr
!sudo apt install tesseract-ocr
!sudo apt install libtesseract-dev
!sudo apt-get install poppler-utils

In [ ]:
%%capture --no-stderr
! pip install -U langchain openai chromadb langchain-experimental langchain_openai nltk pydantic lxml matplotlib chromadb tiktoken
! pip install pillow == 11.1.0
! pip install "unstructured[all-docs]" == 0.17.2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv("/content/.env")